In [30]:
DEVICE = "cpu"

In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("Adilbai/stock-dataset")
df = dataset["train"].to_pandas()

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Ticker", "Date"])

# Choose stock
ticker = "AAPL"
stock_df = df[df["Ticker"] == ticker].copy()

stock_df.reset_index(drop=True, inplace=True)

/tmp/ipykernel_208901/3591124978.py:8: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["Date"] = pd.to_datetime(df["Date"])


In [2]:
df["Ticker"].unique()[:20]
sp_df = df[df["Ticker"] == "^GSPC"][["Date", "Close"]]
sp_df = sp_df.rename(columns={"Close": "SP500_Close"})
stock_df = stock_df.merge(
    sp_df,
    on="Date",
    how="left"
)

In [3]:
import numpy as np
import pandas as pd

K_LIST = [120, 60, 40, 20, 10, 5, 1]  # keep within your history
HORIZON = 10
THRESHOLD = 0.02

# 1) daily returns for each stock
df = df.sort_values(["Ticker", "Date"]).copy()
df["ret_1d"] = df.groupby("Ticker")["Close"].pct_change()

# 2) market proxy return per date (cross-sectional mean)
mkt_ret = df.groupby("Date")["ret_1d"].mean().rename("mkt_ret_1d").reset_index()

# 3) choose one stock (AAPL here)
ticker = "AAPL"
stock_df = df[df["Ticker"] == ticker].sort_values("Date").copy()

# 4) merge market proxy return
stock_df = stock_df.merge(mkt_ret, on="Date", how="left")

# 5) build synthetic market "index price"
# Start at 1.0; cumprod of (1 + return)
stock_df["mkt_price"] = (1.0 + stock_df["mkt_ret_1d"].fillna(0.0)).cumprod()

def k_return(series, k):
    return (series / series.shift(k)) - 1.0

# 6) compute stock returns, market returns, and relative returns X_k
for k in K_LIST:
    stock_df[f"R_stock_{k}"] = k_return(stock_df["Close"], k)
    stock_df[f"R_mkt_{k}"]   = k_return(stock_df["mkt_price"], k)
    stock_df[f"X_{k}"]       = stock_df[f"R_stock_{k}"] - stock_df[f"R_mkt_{k}"]

In [4]:
# We use 1-day relative return X_1 as the “next-day relative return”
# and label y(i)=1 if max_{j=1..10} X_1(i+j) >= 2%
x1 = stock_df["X_1"].values
labels = np.zeros(len(stock_df), dtype=int)

for i in range(len(stock_df) - HORIZON):
    future_window = x1[i+1:i+1+HORIZON]
    labels[i] = int(np.nanmax(future_window) >= THRESHOLD)

stock_df["label"] = labels

# Trim edges explicitly (enough history for max K and enough future for label)
MIN_K = max(K_LIST)
stock_df = stock_df.iloc[MIN_K:-HORIZON].reset_index(drop=True)

In [5]:
feature_cols = [f"X_{k}" for k in K_LIST]
print("Rows:", len(stock_df))
print("NaNs in features:", stock_df[feature_cols].isna().sum().sum())
print(stock_df["label"].value_counts(normalize=True))
print(stock_df[feature_cols].head())

Rows: 1115
NaNs in features: 0
label
0    0.556054
1    0.443946
Name: proportion, dtype: float64
      X_120      X_60      X_40      X_20      X_10       X_5       X_1
0  0.116847  0.011193 -0.010283  0.066499  0.032245 -0.043683  0.002601
1  0.058908 -0.067213 -0.072030 -0.007428 -0.042156 -0.090005 -0.055046
2  0.099304 -0.100685  0.015939  0.014755 -0.050056 -0.053824  0.023682
3  0.075348 -0.074426  0.034212  0.039751 -0.031558 -0.032186  0.006616
4  0.074719 -0.102738 -0.022481  0.002488 -0.059369 -0.046698 -0.022435


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

X = stock_df[feature_cols].copy()
y = stock_df["label"].copy()

X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# Clean infinities (rare here, but safe)
X_train_df = X_train_df.replace([np.inf, -np.inf], np.nan)
X_test_df  = X_test_df.replace([np.inf, -np.inf], np.nan)

# Impute (median)
imputer = SimpleImputer(strategy="median")
X_train_i = imputer.fit_transform(X_train_df)
X_test_i  = imputer.transform(X_test_df)

# Scale
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train_i)
X_test_s  = scaler.transform(X_test_i)

print("Shapes:", X_train_s.shape, X_test_s.shape)
print("NaNs after impute:", np.isnan(X_train_s).sum(), np.isnan(X_test_s).sum())

Shapes: (892, 7) (223, 7)
NaNs after impute: 0 0


In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

def summarize_metrics(metrics_list):
    # metrics_list: list[dict]
    keys = metrics_list[0].keys()
    summary = {}
    for k in keys:
        vals = np.array([m[k] for m in metrics_list], dtype=float)
        summary[k] = {"mean": float(vals.mean()), "std": float(vals.std())}
    return summary


In [12]:
import numpy as np

TRAIN_DAYS = 253
TEST_DAYS = 21
GAP = 10

def make_sliding_windows(df, train_days=253, gap=10, test_days=21):
    windows = []
    start = 0
    n = len(df)
    while start + train_days + gap + test_days <= n:
        train = df.iloc[start : start + train_days]
        test  = df.iloc[start + train_days + gap : start + train_days + gap + test_days]
        windows.append((train, test))
        start += test_days  # shift by 1 month (≈21 trading days)
    return windows

windows = make_sliding_windows(stock_df, TRAIN_DAYS, GAP, TEST_DAYS)
print("Number of windows:", len(windows))


Number of windows: 40


SVM

In [7]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced")
svm.fit(X_train_s, y_train)
preds = svm.predict(X_test_s)

print("SVM accuracy:", accuracy_score(y_test, preds))

SVM accuracy: 0.6098654708520179


In [23]:
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

svm_metrics = []

for train, test in windows:
    X_train_df = train[feature_cols].replace([np.inf, -np.inf], np.nan)
    y_train = train["label"].astype(int).values

    X_test_df = test[feature_cols].replace([np.inf, -np.inf], np.nan)
    y_test = test["label"].astype(int).values

    # Impute per-window
    imputer = SimpleImputer(strategy="median")
    X_train_i = imputer.fit_transform(X_train_df)
    X_test_i  = imputer.transform(X_test_df)

    # Scale per-window
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train_i)
    X_test_s  = scaler.transform(X_test_i)

    svm = SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced")
    svm.fit(X_train_s, y_train)
    preds = svm.predict(X_test_s).astype(int)

    svm_metrics.append(compute_metrics(y_test, preds))

svm_summary = summarize_metrics(svm_metrics)
print("SVM metrics (mean ± std across windows):")
for k, v in svm_summary.items():
    print(f"{k}: {v['mean']:.4f} ± {v['std']:.4f}")

SVM metrics (mean ± std across windows):
accuracy: 0.5476 ± 0.2412
precision: 0.4395 ± 0.3831
recall: 0.4064 ± 0.3686
f1: 0.3588 ± 0.3108


RF

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

rf_metrics = []

for train, test in windows:
    X_train_df = train[feature_cols].replace([np.inf, -np.inf], np.nan)
    y_train = train["label"].astype(int).values

    X_test_df = test[feature_cols].replace([np.inf, -np.inf], np.nan)
    y_test = test["label"].astype(int).values

    # Impute per-window (no leakage)
    imputer = SimpleImputer(strategy="median")
    X_train = imputer.fit_transform(X_train_df)
    X_test  = imputer.transform(X_test_df)

    rf = RandomForestClassifier(
        n_estimators=1000,
        max_depth=5,
        max_features=min(len(feature_cols), 13),
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test).astype(int)

    rf_metrics.append(compute_metrics(y_test, preds))

rf_summary = summarize_metrics(rf_metrics)
print("RF metrics (mean ± std across windows):")
for k, v in rf_summary.items():
    print(f"{k}: {v['mean']:.4f} ± {v['std']:.4f}")

RF metrics (mean ± std across windows):
accuracy: 0.4655 ± 0.2257
precision: 0.3405 ± 0.3618
recall: 0.3270 ± 0.3578
f1: 0.2706 ± 0.2620


LSTM

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def make_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)

def eval_binary(y_true, y_pred):
    return {
        "acc": accuracy_score(y_true, y_pred),
        "prec": precision_score(y_true, y_pred, zero_division=0),
        "rec": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

In [18]:
def preprocess_window(train_df, test_df, feature_cols):
    # keep pandas here; avoid leakage by fitting only on train
    X_train_df = train_df[feature_cols].replace([np.inf, -np.inf], np.nan)
    X_test_df  = test_df[feature_cols].replace([np.inf, -np.inf], np.nan)

    y_train = train_df["label"].astype(int).values
    y_test  = test_df["label"].astype(int).values

    imputer = SimpleImputer(strategy="median")
    X_train_i = imputer.fit_transform(X_train_df)
    X_test_i  = imputer.transform(X_test_df)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train_i)
    X_test_s  = scaler.transform(X_test_i)

    return X_train_s, y_train, X_test_s, y_test

In [19]:
def preprocess_window(train_df, test_df, feature_cols):
    # keep pandas here; avoid leakage by fitting only on train
    X_train_df = train_df[feature_cols].replace([np.inf, -np.inf], np.nan)
    X_test_df  = test_df[feature_cols].replace([np.inf, -np.inf], np.nan)

    y_train = train_df["label"].astype(int).values
    y_test  = test_df["label"].astype(int).values

    imputer = SimpleImputer(strategy="median")
    X_train_i = imputer.fit_transform(X_train_df)
    X_test_i  = imputer.transform(X_test_df)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train_i)
    X_test_s  = scaler.transform(X_test_i)

    return X_train_s, y_train, X_test_s, y_test

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

def build_lstm_model(n_features, lstm_units=64, dropout=0.2):
    model = Sequential([
        LSTM(lstm_units, return_sequences=True, input_shape=(None, n_features)),
        Dropout(dropout),
        LSTM(lstm_units),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy")
    return model

2026-02-08 16:06:28.555154: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-08 16:06:28.555393: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-08 16:06:28.585221: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-08 16:06:29.442922: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

In [21]:
SEQ_LEN = 20
EPOCHS = 30
BATCH_SIZE = 64

lstm_metrics = []
lstm_accs = []

for w_i, (train, test) in enumerate(windows, start=1):
    X_train_s, y_train, X_test_s, y_test = preprocess_window(train, test, feature_cols)

    X_train_seq, y_train_seq = make_sequences(X_train_s, y_train, SEQ_LEN)
    X_test_seq, y_test_seq   = make_sequences(X_test_s, y_test, SEQ_LEN)

    # If a window is too small (rare), skip it safely
    if len(X_train_seq) == 0 or len(X_test_seq) == 0:
        continue

    model = build_lstm_model(n_features=X_train_seq.shape[-1], lstm_units=64, dropout=0.2)

    es = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    model.fit(
        X_train_seq, y_train_seq,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1,
        verbose=0,
        callbacks=[es]
    )

    probs = model.predict(X_test_seq, verbose=0).ravel()
    preds = (probs >= 0.5).astype(int)

    m = eval_binary(y_test_seq, preds)
    lstm_metrics.append(m)
    lstm_accs.append(m["acc"])

print("LSTM mean acc:", float(np.mean(lstm_accs)))
print("LSTM std acc:", float(np.std(lstm_accs)))
print("LSTM mean f1:", float(np.mean([m['f1'] for m in lstm_metrics])))


2026-02-08 16:06:43.683848: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a laye

/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/r

LSTM mean acc: 0.65
LSTM std acc: 0.4769696007084729
LSTM mean f1: 0.25


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def compute_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

def summarize(metrics_list):
    keys = metrics_list[0].keys()
    out = {}
    for k in keys:
        vals = np.array([m[k] for m in metrics_list], dtype=float)
        out[k] = (float(vals.mean()), float(vals.std()))
    return out

def preprocess_window(train_df, test_df, feature_cols):
    # pandas -> numpy, fit transforms ONLY on train (no leakage)
    X_train_df = train_df[feature_cols].replace([np.inf, -np.inf], np.nan)
    X_test_df  = test_df[feature_cols].replace([np.inf, -np.inf], np.nan)

    y_train = train_df["label"].astype(int).values
    y_test  = test_df["label"].astype(int).values

    imputer = SimpleImputer(strategy="median")
    X_train_i = imputer.fit_transform(X_train_df)
    X_test_i  = imputer.transform(X_test_df)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train_i)
    X_test_s  = scaler.transform(X_test_i)

    return X_train_s, y_train, X_test_s, y_test

In [25]:
def make_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)

In [26]:
import torch
import torch.nn as nn
import pennylane as qml

def H_layer(nqubits):
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])

def q_function(x, q_weights, n_class):
    n_dep = q_weights.shape[0]
    n_qub = q_weights.shape[1]

    H_layer(n_qub)
    RY_layer(x)

    for k in range(n_dep):
        entangling_layer(n_qub)
        RY_layer(q_weights[k])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_class)]

class TorchVQC(nn.Module):
    def __init__(self, vqc_depth, n_qubits, n_class):
        super().__init__()
        self.weights = nn.Parameter(0.01 * torch.randn(vqc_depth, n_qubits))
        self.dev = qml.device("default.qubit", wires=n_qubits)
        self.VQC = qml.QNode(q_function, self.dev, interface="torch")
        self.n_class = n_class

    def forward(self, X):
        # X: [B, n_qubits]
        # returns: [B, n_class]
        y_preds = torch.stack([torch.stack(self.VQC(x, self.weights, self.n_class)) for x in X])
        return y_preds.float()

In [27]:
class CustomQLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, vqc_depth):
        super().__init__()
        self.hidden_size = hidden_size
        n_qubits = input_size + hidden_size

        self.input_gate  = TorchVQC(vqc_depth=vqc_depth, n_qubits=n_qubits, n_class=hidden_size)
        self.forget_gate = TorchVQC(vqc_depth=vqc_depth, n_qubits=n_qubits, n_class=hidden_size)
        self.cell_gate   = TorchVQC(vqc_depth=vqc_depth, n_qubits=n_qubits, n_class=hidden_size)
        self.output_gate = TorchVQC(vqc_depth=vqc_depth, n_qubits=n_qubits, n_class=hidden_size)

        self.output_post = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        h_prev, c_prev = hidden
        combined = torch.cat((x, h_prev), dim=1)  # [B, input+hidden]

        i_t = torch.sigmoid(self.input_gate(combined))
        f_t = torch.sigmoid(self.forget_gate(combined))
        g_t = torch.tanh(self.cell_gate(combined))
        o_t = torch.sigmoid(self.output_gate(combined))

        c_t = f_t * c_prev + i_t * g_t
        h_t = o_t * torch.tanh(c_t)

        out = self.output_post(h_t)  # [B, output_size]
        return out, h_t, c_t

class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, cell_module):
        super().__init__()
        self.hidden_size = hidden_size
        self.cell = cell_module

    def forward(self, x, hidden=None):
        # x: [B, T, input_size]
        B, T, _ = x.size()
        if hidden is None:
            h_t = torch.zeros(B, self.hidden_size, device=x.device)
            c_t = torch.zeros(B, self.hidden_size, device=x.device)
        else:
            h_t, c_t = hidden

        outputs = []
        for t in range(T):
            out, h_t, c_t = self.cell(x[:, t, :], (h_t, c_t))
            outputs.append(out.unsqueeze(1))  # [B,1,output_size]

        outputs = torch.cat(outputs, dim=1)  # [B,T,output_size]
        return outputs, (h_t, c_t)

In [32]:
import numpy as np
import torch
import torch.nn as nn

def train_qlstm_one_window(
    X_train_seq, y_train_seq, X_test_seq, y_test_seq,
    hidden_size=4, vqc_depth=1, lr=1e-2, epochs=2,
    batch_size=6, seed=0
):
    # Force CPU to avoid HIP "invalid device function"
    torch.manual_seed(seed)
    np.random.seed(seed)

    Xtr = torch.tensor(X_train_seq, dtype=torch.float32)
    ytr = torch.tensor(y_train_seq, dtype=torch.float32)
    Xte = torch.tensor(X_test_seq, dtype=torch.float32)

    input_size = Xtr.shape[-1]
    output_size = 1

    cell = CustomQLSTMCell(
        input_size=input_size,
        hidden_size=hidden_size,
        output_size=output_size,
        vqc_depth=vqc_depth
    )

    core = CustomLSTM(
        input_size=input_size,
        hidden_size=hidden_size,
        cell_module=cell
    )

    opt = torch.optim.Adam(core.parameters(), lr=lr)
    loss_fn = nn.BCEWithLogitsLoss()

    core.train()
    n = Xtr.shape[0]

    for ep in range(epochs):
        perm = torch.randperm(n)  # CPU permutation (no HIP kernel)
        for i in range(0, n, batch_size):
            idx = perm[i:i+batch_size]
            xb = Xtr[idx]
            yb = ytr[idx]

            opt.zero_grad()
            outputs, _ = core(xb)       # [B,T,1]
            logits = outputs[:, -1, 0]  # [B]
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()

    core.eval()
    with torch.no_grad():
        out_te, _ = core(Xte)
        logits_te = out_te[:, -1, 0]
        probs = torch.sigmoid(logits_te).numpy()
        preds = (probs >= 0.5).astype(int)

    return preds, probs

In [ ]:
qlstm_metrics = []

for w_i, (train, test) in enumerate(windows[:MAX_WINDOWS], start=1):
    X_train_s, y_train, X_test_s, y_test = preprocess_window(train, test, feature_cols)

    X_train_seq, y_train_seq = make_sequences(X_train_s, y_train, SEQ_LEN)
    X_test_seq, y_test_seq   = make_sequences(X_test_s, y_test, SEQ_LEN)

    if len(X_train_seq) == 0 or len(X_test_seq) == 0:
        continue

    preds, probs = train_qlstm_one_window(
        X_train_seq, y_train_seq, X_test_seq, y_test_seq,
        hidden_size=4, vqc_depth=1, lr=1e-2, epochs=4, batch_size=8, seed=42
    )

    m = compute_metrics(y_test_seq, preds)
    qlstm_metrics.append(m)

qlstm_summary = summarize(qlstm_metrics)
print("QLSTM metrics (mean ± std across evaluated windows):")
for k, (mu, sd) in qlstm_summary.items():
    print(f"{k}: {mu:.4f} ± {sd:.4f}")


TypeError: train_qlstm_one_window() got an unexpected keyword argument 'device'

In [ ]:
class VQCClassifier(nn.Module):
    def __init__(self, input_dim, latent_dim=8, vqc_depth=2):
        super().__init__()
        self.fc = nn.Linear(input_dim, latent_dim)
        self.vqc = TorchVQC(vqc_depth=vqc_depth, n_qubits=latent_dim, n_class=1)

    def forward(self, x):
        # x: [B,input_dim]
        z = torch.tanh(self.fc(x))
        logit = self.vqc(z)[:, 0]  # [B]
        return logit

In [ ]:
def print_summary_table(name_to_summary):
    # name_to_summary: dict[str, dict(metric -> (mean,std))]
    metrics = ["accuracy", "precision", "recall", "f1"]
    print("Model".ljust(10), *(m.ljust(20) for m in metrics))
    for name, summ in name_to_summary.items():
        row = [name.ljust(10)]
        for m in metrics:
            mu, sd = summ[m]
            row.append(f"{mu:.4f} ± {sd:.4f}".ljust(20))
        print(*row)

# Example once you have them:
# name_to_summary = {"RF": rf_summary, "SVM": svm_summary, "LSTM": lstm_summary, "QLSTM": qlstm_summary, ...}
# print_summary_table(name_to_summary)